# Setup

In [ ]:
# !pip install torch
# !pip install transformers
# !pip install datasets
# !pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 2

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from torch import nn
from torch import tensor as T
from datasets import load_dataset, Dataset
import sys 
import os
# sys.path.append(os.path.abspath("/content/drive/MyDrive/ELECTRA/Electra"))
from transformers import Trainer, ElectraConfig, ElectraTokenizerFast, ElectraForMaskedLM, ElectraForPreTraining, TrainingArguments
 ## reimplement

# Data

In [ ]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["text"]])
sequence_len = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # Drop the small remainder
    if total_length >= sequence_len:
        total_length = (total_length // sequence_len) * sequence_len
    # Split by chunks of sequence_len.
    result = {
        k: [t[i : i + sequence_len] for i in range(0, total_length, sequence_len)]
        for k, t in concatenated_examples.items()
    }
    #result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
dataset = load_dataset("openwebtext", cache_dir='/content/drive/MyDrive/ELECTRA/Electra/datasets')
tokenizer = ElectraTokenizerFast.from_pretrained(f'google/electra-small-generator')

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset = dataset["train"]

In [ ]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=dataset.column_names,
)

Map (num_proc=4):   0%|          | 0/8013769 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (9125 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4564 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4830 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2205 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

NameError: ignored

In [ ]:
# For testing only
# dataloader = torch.utils.data.DataLoader(lm_dataset, batch_size=16)
# data_collator(next(iter(dataloader)))["attention_mask"][100]

In [ ]:
from dataclasses import dataclass
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union
from collections.abc import Mapping

@dataclass
class ElectraDataCollator():
    tokenizer: ElectraTokenizerFast
    mlm: bool = True
    mlm_probability: float = 0.15
    tf_experimental_compile: bool = False
    return_tensors: str = "pt"

    def __call__(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
      # Handle dict or lists with proper padding and conversion to tensor.
      batch = self.tokenizer.pad(examples, return_tensors="pt")

      # If special token mask has been preprocessed, pop it from the dict.
      special_tokens_mask = batch.pop("special_tokens_mask", None)
  
      batch["masked_ids"], batch["labels"] = self.torch_mask_tokens(
          batch["input_ids"], special_tokens_mask=special_tokens_mask
      )
      batch["is_masked"] = batch["labels"] != -100
      batch.pop("input_ids")
      return batch

    def torch_mask_tokens(self, inputs: Any, special_tokens_mask: Optional[Any] = None) -> Tuple[Any, Any]:
      """
      Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.
      """
      import torch

      labels = inputs.clone()
      # We sample a few tokens in each sequence for MLM training (with probability `self.mlm_probability`)
      probability_matrix = torch.full(labels.shape, self.mlm_probability)
      if special_tokens_mask is None:
          special_tokens_mask = [
              self.tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
          ]
          special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool)
      else:
          special_tokens_mask = special_tokens_mask.bool()

      probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
      masked_indices = torch.bernoulli(probability_matrix).bool()
      labels[~masked_indices] = -100  # We only compute loss on masked tokens

      # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
      indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
      inputs[indices_replaced] = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)

      # 10% of the time, we replace masked input tokens with random word
      indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
      random_words = torch.randint(len(self.tokenizer), labels.shape, dtype=torch.long)
      inputs[indices_random] = random_words[indices_random]

      # The rest of the time (10% of the time) we keep the masked input tokens unchanged
      return inputs, labels

# Electra Model

In [ ]:
## ELECTRA Model, input: generator and discriminator
## Joinly trained both models
class ELECTRAModel(nn.Module):
  def __init__(self, generator, discriminator, pad_token_id, dis_loss_weight= 50.0):
    super().__init__()
    self.generator, self.discriminator, self.pad_token_id = generator, discriminator, pad_token_id
    self.gumbel_dist = torch.distributions.gumbel.Gumbel(torch.tensor(0., device='cuda:0'), torch.tensor(1., device='cuda:0'))
    self.dis_loss_weight = dis_loss_weight
    self.gen_loss_fc = nn.CrossEntropyLoss() #maybe Flatten
    self.dis_loss_fc = nn.BCEWithLogitsLoss()

  def forward(self, masked_ids, attention_mask, token_type_ids, is_masked, labels):
    # masked_ids: masked input ids (Tensor[int]): (B, L)
    # attention_mask: attention_mask from data collator (Tensor[int]): (B, L)
    # token_type_ids: token_type_ids of the tokenizer
    # is_masked: whether the position is get masked (Tensor[boolean]): (B, L)
    # labels: (Tensor[int]): (batch_size, seq_length) -100 for the unmasked

    # Feed into generator
    gen_logits = self.generator(masked_ids, attention_mask, token_type_ids).logits # (B, L, vocab size)
    masked_gen_logits = gen_logits[is_masked,:] 

    with torch.no_grad():
      # gumble arg-max to have tokenized predicted word
      pred_toks = self.gumbel_softmax(masked_gen_logits)
      # inputs for discriminator
      gen_ids = masked_ids.clone() 
      gen_ids[is_masked] = pred_toks #(B, L)
      # labels for discriminator
      is_replaced = is_masked.clone() 
      is_replaced[is_masked] = (pred_toks != labels[is_masked]) #(B, L)

    # Feed into discrminator
    dis_logits = self.discriminator(gen_ids, attention_mask, token_type_ids).logits # (B, L, vocab size)

    # Loss function of Electra
    gen_loss = self.gen_loss_fc(masked_gen_logits.float(), labels[is_masked])
    # Discriminator Loss
    dis_logits = dis_logits.masked_select(attention_mask==1) 
    is_replaced = is_replaced.masked_select(attention_mask==1) 
    disc_loss = self.dis_loss_fc(dis_logits.float(), is_replaced.float())
    loss = gen_loss + disc_loss * self.dis_loss_weight

    return {
        "loss": loss,
        "masked_gen_logits": masked_gen_logits, 
        "gen_logits": gen_logits,
        "dis_logits": dis_logits
        }
      
  def gumbel_softmax(self, logits):
    return (logits.float() + self.gumbel_dist.sample(logits.shape)).argmax(dim=-1)
    

# Train

In [ ]:
disc_config = ElectraConfig.from_pretrained(f'google/electra-small-discriminator')
gen_config = ElectraConfig.from_pretrained(f'google/electra-small-generator')
# tokenizer = ElectraTokenizerFast.from_pretrained(f'google/electra-small-generator')
# dataset = load_dataset("openwebtext", cache_dir='/content/drive/MyDrive/ELECTRA/Electra/datasets')

In [ ]:
#dataset['train']

Dataset({
    features: ['text'],
    num_rows: 8013769
})

In [ ]:
#dataset['train'][0]["text"]

'Port-au-Prince, Haiti (CNN) -- Earthquake victims, writhing in pain and grasping at life, watched doctors and nurses walk away from a field hospital Friday night after a Belgian medical team evacuated the area, saying it was concerned about security.\n\nThe decision left CNN Chief Medical Correspondent Sanjay Gupta as the only doctor at the hospital to get the patients through the night.\n\nCNN initially reported, based on conversations with some of the doctors, that the United Nations ordered the Belgian First Aid and Support Team to evacuate. However, Belgian Chief Coordinator Geert Gijs, a doctor who was at the hospital with 60 Belgian medical personnel, said it was his decision to pull the team out for the night. Gijs said he requested U.N. security personnel to staff the hospital overnight, but was told that peacekeepers would only be able to evacuate the team.\n\nHe said it was a "tough decision" but that he accepted the U.N. offer to evacuate after a Canadian medical team, also

In [ ]:
SEED = 203
generator = ElectraForMaskedLM(gen_config)
discriminator = ElectraForPreTraining(disc_config)
discriminator.electra.embeddings = generator.electra.embeddings
generator.generator_lm_head.weight = generator.electra.embeddings.word_embeddings.weight
pad_token_id = tokenizer.pad_token_id
# Data Collator
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = ElectraDataCollator(tokenizer=tokenizer, mlm_probability=0.15)
training_args = TrainingArguments(
        output_dir= "/content/drive/MyDrive/ELECTRA/Electra/models",
        do_train=True,
        do_eval=True,
        per_device_train_batch_size=128,
        per_device_eval_batch_size=128,
        learning_rate=5e-4,
        logging_dir= "/content/drive/MyDrive/ELECTRA/Electra/logs",
        report_to= "wandb",
        remove_unused_columns=False,
        save_total_limit=2,
        save_strategy = "no",
        load_best_model_at_end=False,
        warmup_steps=10000,
        weight_decay=0.01,
        save_steps=1000,
        adam_epsilon=1e-6,
        max_steps=62500,
        seed=SEED,
    )

In [ ]:
from transformers import AdamW
model = ELECTRAModel(generator, discriminator, pad_token_id)
trainer = Trainer(
    model =model,
    args = training_args,
    train_dataset = lm_dataset,
    data_collator = data_collator,
    )

NameError: ignored

In [ ]:
trainer.train()
trainer.save_model("/content/drive/MyDrive/ELECTRA/Electra/pretrained")